<a href="https://colab.research.google.com/github/10Vaibhav/Reference-for-HMI/blob/main/HyperSpectralClassification_2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install earthpy

# Import Libraries

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import earthpy.plot as ep
import seaborn as sns
import earthpy.spatial as es

import plotly.graph_objects as go
import plotly.express as px

from scipy.io import loadmat

import pandas as pd

from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, confusion_matrix, classification_report)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, MaxPooling2D, Dropout, Flatten, Conv2D, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from tqdm import tqdm
from numpy.random import seed
from time import time

seed(11)

In [3]:
## GLOBAL VARIABLES
test_ratio = 0.3
windowSize = 3

# Required Functions

In [4]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState, stratify=y)

      return X_train, X_test, y_train, y_test

In [5]:
# if you want to have inputs and outputs, map size should be the same then we need zero padding
def padWithZeros(X, margin=2):
  newX = np.zeros((X.shape[0] + 2*margin, X.shape[1] + 2*margin, X.shape[2]))
  x_offset = margin
  y_offset = margin
  newX[x_offset:X.shape[0]+x_offset, y_offset: X.shape[1] + y_offset, :] = X
  return newX

In [6]:
def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
  margin = int((windowSize -1)/ 2)
  zeroPaddedX = padWithZeros(X, margin=margin)
  # split patches
  patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
  patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
  patchIndex = 0
  for r in range(margin, zeroPaddedX.shape[0] - margin):
    for c in range(margin, zeroPaddedX.shape[1]- margin):
      patch = zeroPaddedX[r - margin:r + margin + 1, c - margin: c + margin + 1]
      patchesData[patchIndex, :, :, :] = patch
      patchesLabels[patchIndex] = y[r-margin, c-margin]
      patchIndex = patchIndex + 1
  if removeZeroLabels:
    patchesData = patchesData[patchesLabels>0, :, :, :]
    patchesLabels = patchesLabels[patchesLabels>0]
    patchesLabels -= 1
  return patchesData, patchesLabels

# Load Data

In [7]:
# X data (1096, 715, 102) Hyperspectral satellite (1096*715, 102)
data = loadmat('Pavia.mat')['pavia']

In [8]:
gt = loadmat('Pavia_gt.mat')['pavia_gt']

In [9]:
print(f'Data Shape: {data.shape[:-1]} \n Number of Bands: {data.shape[-1]}')

Data Shape: (1096, 715) 
 Number of Bands: 102


In [10]:
x,y = createImageCubes(data, gt, windowSize=windowSize)
x.shape, y.shape

((148152, 3, 3, 102), (148152,))

In [12]:
x_train, x_test, y_train, y_test = splitTrainTestSet(x, y, test_ratio)

# Data Visualization
